In [0]:
from scipy.misc import imread
import numpy as np
import pandas as pd
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Activation,BatchNormalization, Dense
from keras.optimizers import Adam,SGD

In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

datagen=ImageDataGenerator()
img=datagen.flow_from_directory('drive/My Drive/train1')

Found 3005 images belonging to 3 classes.


In [0]:

img_test=datagen.flow_from_directory('drive/My Drive/test')

Found 150 images belonging to 3 classes.


In [0]:
model=Sequential()
model.add(Conv2D(16,3, activation='relu',input_shape=(256,256,3)))
model.add(Conv2D(32,3, activation='relu'))
model.add(Conv2D(32,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(48,3, activation='relu'))
model.add(Conv2D(48,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64,3, activation='relu'))
model.add(Conv2D(64,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(72,3, activation='relu'))
model.add(Conv2D(72,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(72,3, activation='relu'))
model.add(Conv2D(72,3, activation='relu'))
model.add(Conv2D(80,3, activation='relu'))
model.add(Conv2D(3,6))
model.add(Flatten())
model.add(Activation('softmax'))


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 254, 254, 16)      448       
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 252, 252, 32)      4640      
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 250, 250, 32)      9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 125, 125, 32)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 125, 125, 32)      128       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 123, 123, 48)      13872     
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 121, 121, 48)      20784     
__________

In [0]:
model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
m=model.fit_generator(img, epochs=10,steps_per_epoch=500,validation_data=img_test,validation_steps=640)

Epoch 1/10
500/500 [==============================] - 519s 1s/step - loss: 1.0009 - acc: 0.4821 - val_loss: 1.2062 - val_acc: 0.5000
Epoch 2/10
500/500 [==============================] - 513s 1s/step - loss: 0.3916 - acc: 0.8440 - val_loss: 2.2230 - val_acc: 0.5733
Epoch 3/10
500/500 [==============================] - 521s 1s/step - loss: 0.0632 - acc: 0.9802 - val_loss: 1.7160 - val_acc: 0.6867
Epoch 4/10
500/500 [==============================] - 516s 1s/step - loss: 0.0368 - acc: 0.9902 - val_loss: 2.2126 - val_acc: 0.6400
Epoch 5/10
500/500 [==============================] - 516s 1s/step - loss: 0.0243 - acc: 0.9922 - val_loss: 2.6185 - val_acc: 0.6667
Epoch 6/10
500/500 [==============================] - 515s 1s/step - loss: 0.0206 - acc: 0.9946 - val_loss: 2.5517 - val_acc: 0.6800
Epoch 7/10
500/500 [==============================] - 513s 1s/step - loss: 0.0072 - acc: 0.9980 - val_loss: 2.7261 - val_acc: 0.6600
Epoch 8/10
500/500 [==============================] - 516s 1s/step - 

In [0]:
scores = model.evaluate_generator(img_test, steps=1000, verbose=1)

1000/1000 [==============================] - 340s 340ms/step


In [0]:
print("Accuracy is %s" %(scores[1]*100))

Accuracy is 64.68666661123434


In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
Y_pred = model.predict_generator(img_test, steps=5)
y_pred = np.argmax(Y_pred,axis=1)

In [0]:

print("Confusion Metrix")
print(metrics.confusion_matrix(img_test.classes, y_pred))
print("Classification Report")
print(metrics.classification_report(img_test.classes, y_pred))



Confusion Metrix
[[14 18 18]
 [23  7 20]
 [13 14 23]]
Classification Report
              precision    recall  f1-score   support

           0       0.28      0.28      0.28        50
           1       0.18      0.14      0.16        50
           2       0.38      0.46      0.41        50

   micro avg       0.29      0.29      0.29       150
   macro avg       0.28      0.29      0.28       150
weighted avg       0.28      0.29      0.28       150



In [0]:
from keras.models import model_from_yaml

In [0]:
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
  yaml_file.write(model_yaml)
model.save_weights("model.h5")
print("Saved model")

Saved model
